In [1]:
import syft as sy
import torch
import torch.nn as nn
import time
import torch.autograd as autograd

hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
james = sy.VirtualWorker(hook, id="james")


In [2]:
from syft.frameworks.torch.tensors.interpreters import gradients
from syft.frameworks.torch.tensors.interpreters.gradients_core import GradFunc
from syft.frameworks.torch.tensors.interpreters.gradients_core import apply_dim_transformations

In [3]:
from functools import wraps

In [18]:
def declare_backward_op(attr):
    name = attr.__name__[:-1]
    print('^^ declare',name)
    sy.frameworks.torch.tensors.interpreters.autograd.backward_funcs[name] = attr
    return attr

sy.autograd = declare_backward_op

def manual_overload(attr):
    
    @wraps(attr)
    def overloaded_func(*args, **kwargs):
        """
        Operate the hooking
        """
        child = False
        if child:
            tensor = args[0] if not isinstance(args[0], tuple) else args[0][0]
            cmd = attr.__name__

            new_args, new_kwargs, new_type = sy.frameworks.torch.hook_args.hook_function_args(
                cmd, args, kwargs
            )

            response = attr(*new_args, **new_kwargs)
            
            

            response = sy.frameworks.torch.hook_args.hook_response(
                cmd, response, wrap_type=sy.AutogradTensor, wrap_args=tensor.get_class_attributes()
            )
        else:
            response = attr(*args, **kwargs)
        
        grad_fn = sy.frameworks.torch.tensors.interpreters.autograd.get_backward_func(attr.__name__)
        if grad_fn is not None:
            response.child.grad_fn = grad_fn(*args, **kwargs)
            response.child.grad_fn.result = response.child
        else:
            print('failed to get back func for ', attr.__name__)
                
        return response

    return overloaded_func

In [19]:
@sy.autograd
class add2_(GradFunc):
    def __init__(self, self_, other):
        super().__init__(self, self_, other)
        self.self_ = self_
        self.other = other

    def gradient(self, grad):
        print('**custom backward')
        grad_self = grad.copy()
        grad_other = grad.copy()

        print(grad_self, grad_other)
        return (grad_self, grad_other)

@manual_overload
def add2(x, y):
    print("** Custom forward called")
    return x + y

x = torch.tensor([1.5], requires_grad=True)
x = x.send(alice, local_autograd=True).child
y = torch.tensor([1.5], requires_grad=True)
y = y.send(alice, local_autograd=True).child

#add2 = MyFunction.apply

z = add2(x, y)

print('?')
print(z.grad_fn)
print('---before back')
z.backward()
print('---')
x.grad

^^ declare add2
** Custom forward called
add ok
> grad_fn is none
> grad_fn None
>?  AutogradTensor>[PointerTensor | me:46653912145 -> alice:60861543499]
> accumul creater
> grad_fn None
>?  AutogradTensor>[PointerTensor | me:70344948923 -> alice:93824877373]
> accumul creater
++ [None, Accumulate, Accumulate]
AddBackward => (Accumulate, Accumulate)
add2 ok
> grad_fn is none
> grad_fn None
>?  AutogradTensor>[PointerTensor | me:46653912145 -> alice:60861543499]
> accumul creater
> grad_fn None
>?  AutogradTensor>[PointerTensor | me:70344948923 -> alice:93824877373]
> accumul creater
++ [None, Accumulate, Accumulate]
add2_ => (Accumulate, Accumulate)
?
AddBackward
---before back
mul ok
> grad_fn is none
> grad_fn AddBackward
>?  AutogradTensor>[PointerTensor | me:36510725982 -> alice:95213759436]
> no AddBackward
> grad_fn is none
++ [None, AddBackward, None]
MulBackward => (AddBackward,)
add ok
> grad_fn is none
> grad_fn MulBackward
>?  AutogradTensor>[PointerTensor | me:22993925352 -

[PointerTensor | me:74510549920 -> alice:66546187908]

In [9]:
x = torch.tensor([1.5], requires_grad=True)
x = x.send(alice, local_autograd=True)
y = torch.tensor([1.5], requires_grad=True)
y = y.send(alice, local_autograd=True)

z = x + y
print('---before back')
z.backward()

x.grad.get()

__add__ fail
add ok
> grad_fn is none
> grad_fn None
>?  AutogradTensor>[PointerTensor | me:33261298050 -> alice:41528094981]
> accumul creater
> grad_fn None
>?  AutogradTensor>[PointerTensor | me:76213256870 -> alice:18969709600]
> accumul creater
++ [None, Accumulate, Accumulate]
AddBackward => (Accumulate, Accumulate)
---before back
mul ok
> grad_fn is none
> grad_fn AddBackward
>?  AutogradTensor>[PointerTensor | me:44573709321 -> alice:89079664745]
> no AddBackward
> grad_fn is none
++ [None, AddBackward, None]
MulBackward => (AddBackward,)
add ok
> grad_fn is none
> grad_fn MulBackward
>?  AutogradTensor>[PointerTensor | me:50406920954 -> alice:44993426422]
> no MulBackward
> grad_fn is none
++ [None, MulBackward, None]
AddBackward => (MulBackward,)
add ok
> grad_fn is none
> grad_fn AddBackward
>?  AutogradTensor>[PointerTensor | me:981925666 -> alice:92874071320]
> no AddBackward
> grad_fn is none
++ [None, AddBackward, None]
AddBackward => (AddBackward,)
add ok
> grad_fn is n

tensor([1.], requires_grad=True)

In [25]:
x.child

__class__ fail
_ipython_canary_method_should_not_exist_ fail
_ipython_display_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_mimebundle_ fail
__class__ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_html_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_markdown_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_svg_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_png_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_pdf_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_jpeg_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_latex_ fail
__class__ fail
__class__ fail
_ipython_canary_method_should_not_exist_ fail
_repr_json_ fail
__class__ fail
__class__ fail
_ipython_canary_method_sho

AutogradTensor>[PointerTensor | me:94136685418 -> alice:55533850958]

In [17]:
class MyFunction(torch.autograd.Function):
    def forward(self, x, y):
        print(">>> Custom forward called")
        return x + y
    
    def backward(self, grad_out):
        grad_input = grad_out.clone()
        print('>>> Custom backward called!')
        return grad_input
    

In [14]:
@sy.autograd
class MyFunction(torch.autograd.Function):
    def forward(self, x):
        print("Custom forward called")
        return x
    
    def backward(self, grad_out):
        grad_input = grad_out.clone()
        print('Custom backward called!')
        return grad_input


x = torch.tensor([1.5], requires_grad=True)
x = x.send(alice, local_autograd=True)

relu = MyFunction.apply

y = relu(x)
print('---')
y.backward()

on fail
Custom forward called
---
backward fail
mul fail
child fail
get_class_attributes fail
on fail
add fail
child fail
get_class_attributes fail
on fail
grad_fn fail


ValueError: The gradient for one of the command you used was not found. Check gradients.py to see if it's missing.

In [2]:
import random

In [9]:
ts = []
n=50
for i in range(n):
    t = torch.tensor([random.random()])
    x = t.fix_prec().share(bob, alice, crypto_provider=james)
    x_sh = x.child.child.child
    x_sh.child = {
        'alice': random.random(),
        'bob': random.random()
    }

    start_time = time.time()
    y = x + x
    t = time.time() - start_time
    ts.append(t)
    
print(sum(ts)/n)

6.389617919921875e-07


In [9]:
ts = []
n=50
for i in range(n):
    t = torch.tensor([random.random()])
    x = t.fix_prec().share(bob, alice, crypto_provider=james)
    x_sh = x.child.child.child
    x_sh.child = {
        'alice': random.random(),
        'bob': random.random()
    }

    start_time = time.time()
    y = x + x
    t = time.time() - start_time
    ts.append(t)
    
print(sum(ts)/n)

0.00013466835021972657


In [ ]:
0.001093149185180664
0.0008411407470703125
0.0008440017700195312
0.0008211135864257812
0.0008141994476318359
0.0013489723205566406
0.0008919239044189453
0.0008268356323242188
0.0011949539184570312
0.0008399486541748047

In [3]:
a = (
    torch.tensor([[3.0, 2], [-1, 2]], requires_grad=True)
        .fix_prec()
        .share(alice, bob, crypto_provider=james)
)
sy.AutogradTensor().on(a)
b = 2

c = a / b

c.get().child.float_prec()

tensor([[ 1.5000,  1.0000],
        [-0.5000,  1.0000]])

In [9]:
backward_one = True
a = (
    torch.tensor([[3.0, 2], [-1, 2]], requires_grad=True)
        .fix_prec()
        .share(alice, bob, crypto_provider=james)
)
b = 2

a = sy.AutogradTensor().on(a)

a_torch = torch.tensor([[3.0, 2], [-1, 2]], requires_grad=True)
b_torch = 2

c = a / b
c_torch = a_torch / b_torch

ones = torch.ones(c.shape).fix_prec().share(alice, bob, crypto_provider=james)
ones = sy.AutogradTensor().on(ones)
c.backward(ones if backward_one else None)
c_torch.backward(torch.ones(c_torch.shape))

print(c.get().child.float_prec())
print(a.grad.get().float_prec())
print(a_torch.grad)

tensor([[-2.3058e+15, -2.3058e+15],
        [ 2.3058e+15, -2.3058e+15]])
tensor([[-2.3058e+15, -2.3058e+15],
        [-2.3058e+15, -2.3058e+15]])
tensor([[0.5000, 0.5000],
        [0.5000, 0.5000]])


In [2]:
bob._objects
(x_sh + x_sh).get()

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:43557234540 -> alice:78963339608]
	-> (Wrapper)>[PointerTensor | me:55141822262 -> bob:55948682756]
	*crypto provider: james*

In [5]:
t = torch.tensor([1, 2, 3, 4.0])
ptr = t.send(james)

x = ptr.fix_prec().share(bob, alice)

y = x + x

y = y.get().get().float_prec()
assert (y == (t + t)).all()

In [2]:
x_sh + x_sh

KeyError: 'Object "85571574261" not found on worker!!!You just tried to interact with an object ID:85571574261 on <VirtualWorker id:alice #objects:0> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn

import syft as sy
hook = sy.TorchHook(torch)
worker = sy.VirtualWorker(hook, id="alice")

class MyFunction(torch.autograd.Function):
    def forward(self, x):
        print("Custom forward called")
        return x
    
    def backward(self, grad_out):
        grad_input = grad_out.clone()
        print('Custom backward called!')
        return grad_input


x = torch.tensor([1.5], requires_grad=True)
x = x.send(worker, local_autograd=True)

y = MyFunction(x)
print(y)
y.backward()

In [1]:
import torch
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import syft as sy
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
import sys

alice = sy.VirtualWorker(hook, id="worker")
x = torch.tensor([1.])
x_ptr = x.send(alice)
x_fp = x_ptr.fix_prec()

AttributeError: 'numpy.ndarray' object has no attribute 'owner'

In [1]:
import torch
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import syft as sy
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
import sys

In [2]:
import numpy as np
jon = james

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn

import syft as sy
hook = sy.TorchHook(torch)
worker = sy.VirtualWorker(hook, id="alice")

class MyFunction(torch.autograd.Function):
    def forward(self, x):
        print("Custom forward called")
        return x
    
    def backward(self, grad_out):
        grad_input = grad_out.clone()
        print('Custom backward called!')
        return grad_input


x = torch.tensor([1.5], requires_grad=True)
x = x.send(worker, local_autograd=True)

y = MyFunction(x)
print(y)
y.backward()

TypeError: backward() missing 1 required positional argument: 'grad_out'

In [3]:
wks = [ sy.VirtualWorker(hook, id="w#%d" % i) for i in range(2) ]

In [ ]:
wks = [alice, bob]

In [4]:
t = torch.zeros(3,3)
t = t.fix_prec().share(*wks, crypto_provider=james)
t = t * 2
t = t.get()
t.float_prec()

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [3]:
x = th.tensor([1., 2., 3.]).fix_precision().send(alice)
x

(Wrapper)>[PointerTensor | me:15619336534 -> alice:67580133165]

In [4]:
x = x.share(alice, bob, crypto_provider=jon)
x

(Wrapper)>[PointerTensor | me:42441947787 -> alice:30145459299]

In [5]:
x.get()

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:28552406043 -> alice:15863354938]
	-> (Wrapper)>[PointerTensor | me:94183195016 -> bob:44685349841]
	*crypto provider: james*

In [7]:
x_ptr = th.tensor([1., 2., 3.]).fix_precision()#.send(jon)
y_ptr = th.tensor([4., 5., 6.]).fix_precision()#.send(jon)

x_shared = x_ptr.share(alice, bob, crypto_provider=jon)#.get()
y_shared = y_ptr.share(alice, bob, crypto_provider=jon)#.get()

print(x_shared)
print(y_shared)
secure_sum = x_shared + y_shared
print(secure_sum)
# (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
# 	-> (Wrapper)>[PointerTensor | me:92050228605 -> alice:43660865053]
# 	-> (Wrapper)>[PointerTensor | me:15127770581 -> bob:67436046413]
# 	*crypto provider: jon*

secure_sum_squared = secure_sum * secure_sum
print(secure_sum_squared)
# (Wrapper)>FixedPrecisionTensor>None

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:25273663339 -> alice:30559571619]
	-> (Wrapper)>[PointerTensor | me:8099471303 -> bob:80102907237]
	*crypto provider: james*
(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:84139256652 -> alice:23631550677]
	-> (Wrapper)>[PointerTensor | me:68636278393 -> bob:59842209046]
	*crypto provider: james*
(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:84684315617 -> alice:32766993872]
	-> (Wrapper)>[PointerTensor | me:924594343 -> bob:49059878250]
	*crypto provider: james*
(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:29884658376 -> alice:71494212731]
	-> (Wrapper)>[PointerTensor | me:55438291069 -> bob:8078723432]
	*crypto provider: james*


In [ ]:
x_ptr = th.tensor([1., 2., 3.]).fix_precision()
y_ptr = th.tensor([4., 5., 6.]).fix_precision()

x_shared = x_ptr.share(alice, bob, crypto_provider=jon)#.get()
y_shared = y_ptr.share(alice, bob, crypto_provider=jon)#.get()

secure_sum = x_shared + y_shared
print(secure_sum)

secure_sum_squared = secure_sum * secure_sum
print(secure_sum_squared)

In [91]:
q = 100
base = 2
prec = 3
C = 10 #base ** prec
print(q, C)

100 10


In [110]:
def trunc(x1, x2):
    return x1/C, q - (q - x2)/C

In [111]:
trunc(70, 70) # 40 -> 4

(7.0, 97.0)

In [106]:
def _trunc(x1, x2):
    if x1 > q/2:
        x1 = (x1 - q)
    if x2 > q/2:
        x2 = (x2 - q)
    x1 /= C
    x2 /= C
    return x1, x2

In [109]:
_trunc(70, 70) # 40 -> 4

(-3.0, -3.0)

In [87]:
def enc(x):
    return np.round(x * C) % q

def dec(x):
    if x > q/2:
        x = (x - q)
    # Convert to float
    return x / C

def add(x, y):
    z = x + y
    return z % q

def mul(x, y):
    z = (x * y) % q
    r = dec(z)
    print(r)
    return r

In [88]:
q/(C*2)

64.0625

In [90]:
a, b = -1, 4
_a = enc(a)
_b = enc(b)
print(dec(add(_a, _b)))
print(dec(mul(_a, _b)))

3.0
-32.0
-4.0


In [48]:
1025 - 8 * 8 *4

769

In [12]:
a = -2
b = -3
a * b

6

In [13]:
def mul(x, y):
    return ((x * y) % q)/C

In [14]:
mul(enc(a), enc(b))

48.0

In [15]:
def dec(x):
    if x > q/2:
        x = (x - q)
    # Convert to float
    return x / C

In [16]:
dec(8)

1.0

In [17]:
dec(1017)

-1.0

In [18]:
dec(mul(enc(a), enc(b)))

6.0

In [19]:
a = 2
b = 3

In [20]:
dec(mul(enc(a), enc(b)))

6.0

In [30]:
a = -2
b = 1
enc(a), enc(b)

(1009, 8)

In [34]:
(enc(a) * enc(b)) % q

897

In [35]:
dec(897) / C

-2.0

In [31]:
mul(enc(a), enc(b))

112.125

In [33]:
enc(-3*8)

833

In [37]:
enc(1)

8

In [140]:
def enc(x):
    return (x * C) % field
def dec(x):
    ## wrap in [0, q]
    #x = x % q
    # wrap in [-q/2, q/2]
    if x > q/(2):
        x = (x - q)
    # Convert to float
    return x / C
def add(x, y):
    return (x + y) % q
def mul(x, y):
    return np.round((x * y) / C) % q

In [141]:
a, b = -1, -1

In [142]:
mul(enc(a), enc(b))

57.0

In [143]:
dec(mul(enc(a), enc(b)))

7.125

In [58]:
x = torch.tensor([1.])
p = x.send(bob)

In [65]:
%%time
l = []
x = torch.tensor([1.])
p = x.send(bob)
for i in range(10000):
    if hasattr(p, 'child'):
        l.append(p.child)

CPU times: user 3.36 ms, sys: 76 µs, total: 3.44 ms
Wall time: 3.4 ms


In [66]:
%%time
l = []
x = torch.tensor([1.])
p = x.send(bob)
for i in range(10000):
    try:
        l.append(p.child)
    except KeyError:
        pass
        

CPU times: user 1.94 ms, sys: 52 µs, total: 2 ms
Wall time: 1.96 ms


In [63]:
%%time
l = []
for i in range(10000):
    x = torch.tensor([1.])
    p = x.send(bob)
    if hasattr(p, 'child'):
        l.append(p.child)

CPU times: user 1.97 s, sys: 208 ms, total: 2.18 s
Wall time: 2.27 s


In [64]:
%%time
l = []
for i in range(10000):
    x = torch.tensor([1.])
    p = x.send(bob)
    try:
        l.append(p.child)
    except KeyError:
        pass
        

CPU times: user 1.88 s, sys: 22 ms, total: 1.9 s
Wall time: 1.91 s


In [34]:
dec(mul(a_fp, b_fp)) 

7.625

In [19]:
a * (b * C + q) 

110

In [23]:
a * b * C + a * q  ,   a * b * C

(110, -16)

In [27]:
dec((a * (b * C + q)) % q)

5.875

In [48]:
mul(x_fp, y_fp) % (field * base ** prec)

119.25

In [45]:
dec(mul(x_fp, y_fp)), x * y

(14.90625, -2.76)

In [47]:
2.25 * (6.625 - field / base ** prec)

-2.8125

In [2]:
x = th.tensor([1.])
x_fp = x.fix_precision()

r_fp = x_fp / 5
r = r_fp.float_precision()
print(r)
assert r_fp.float_precision() == x / 5

tensor([0.2000])


In [4]:
x = th.tensor([1.])
x_fp = x.fix_precision()

r_fp = x_fp + 10
r = r_fp.float_precision()
print(r)
assert r == x + 10

r_fp = x_fp - 7
r = r_fp.float_precision()
print(r)
assert r == x - 7

r_fp = x_fp * 2
assert r_fp.float_precision() == x * 2

r_fp = x_fp / 5
assert r_fp.float_precision() == x / 5

tensor([11.])
tensor([-6.])


In [5]:
x = th.tensor([2.])
x_sh = x.fix_precision().share(alice, bob, crypto_provider=james)

r_sh = x_sh + 10
assert r_sh.get().float_prec() == x + 10

r_sh = x_sh - 7
assert r_sh.get().float_prec() == x - 7

r_sh = x_sh * 2
assert r_sh.get().float_prec() == x * 2

r_sh = x_sh / 2
assert r_sh.get().float_prec() == x / 2

In [2]:
x = th.tensor([1.])
y = th.tensor([2.])
z = torch.mul(x, y)
z.backward()
z.grad_fn

NameError: name 'th' is not defined

In [6]:
x = th.tensor([1])
x = x.share(alice, bob, crypto_provider=james).child
y = th.tensor([2])
y = y.share(alice, bob, crypto_provider=james).child

In [7]:
z = torch.mul(x, y)

_public_mul


In [8]:
z
z.virtual_get()

tensor([2])

In [9]:
z.grad_fn

AttributeError: 'AdditiveSharingTensor' object has no attribute 'grad_fn'

In [5]:
z.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [4]:
x = th.tensor([1, 2])
x.wrap()

(Wrapper)>tensor([1, 2])

In [16]:
x = th.tensor([1, 6])
xsh = x.share(alice, bob, crypto_provider=james).child

In [17]:
xsh

[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:68316565755 -> alice:58571883017]
	-> (Wrapper)>[PointerTensor | me:71738605268 -> bob:35277627239]
	*crypto provider: james*

In [25]:
args = []
kwargs = {'dim':0}
self = xsh

summ = self.sum(*args, **kwargs)

 # We need to know how many input values are used for each
# output value to divide
dims_to_reduce = args if args else range(self.dim())
print(dims_to_reduce)

div = 1
for i, s in enumerate(self.shape):
    if i in dims_to_reduce:
        div *= s

z = summ // div
z.get()

range(0, 1)


tensor(4)

In [3]:
other_shares

{'alice': (Wrapper)>[PointerTensor | me:20238619441 -> alice:32218935864],
 'bob': (Wrapper)>[PointerTensor | me:28256774228 -> bob:52232125509]}

In [2]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# Instantiate the model
model = Net()

# The data itself doesn't matter as long as the shape is right
mock_data = torch.zeros(1, 2)

# Create a jit version of the model
traced_model = torch.jit.trace(model, mock_data)

type(traced_model)

torch.jit.TopLevelTracedModule

In [4]:
isinstance(traced_model, torch.jit.ScriptModule)

True

In [11]:
@torch.jit.script
def loss_fn(real, pred):
    return ((real.float() - pred.float()) ** 2).mean()

In [14]:
loss_fn

In [15]:
isinstance(loss_fn, torch.jit.ScriptModule)

False

In [13]:
type(loss_fn)

torch._C.Function

In [10]:
l = [1, 2, 3]
l.drop(4)
l

AttributeError: 'list' object has no attribute 'drop'

In [6]:
x = torch.Tensor([-1., -2, 3, 4])
x = x.fix_precision()
print(x)
x_priv = x.share(alice, bob, crypto_provider=james)
print(x)
x = torch.max(x)
x.get()

(Wrapper)>FixedPrecisionTensor>tensor([-1000, -2000,  3000,  4000])
(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:82835526832 -> alice:82205771933]
	-> (Wrapper)>[PointerTensor | me:60559260313 -> bob:82962870003]
	*crypto provider: james*


GetNotPermittedError: 

In [2]:
t = torch.tensor([1., 2, 3, 4]).send(alice)

In [3]:
t.get()

GetNotPermittedError: 

In [2]:
t = torch.tensor([1., 2, 3, 4]).fix_precision().share(alice, bob, crypto_provider=james)

In [3]:
torch.add(t, t)

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:83306198803 -> alice:76719578477]
	-> (Wrapper)>[PointerTensor | me:91666611547 -> alice:23452685639]
	*crypto provider: james*

In [8]:
t.child.child.child.torch.nn.__module__

'torchtorch'

In [4]:
t.child.child.child.torch.stack.__module__

'syft.frameworks.torch.overload_torch'

In [4]:
torch.stack([t, t])

syft.frameworks.torch.tensors.interpreters.additive_shared stack


(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:87013070637 -> alice:5270771564]
	-> (Wrapper)>[PointerTensor | me:54738216108 -> bob:7537631574]
	*crypto provider: james*

In [4]:
t = torch.tensor([1., 2, 3, 4]).fix_precision().share(alice, bob, crypto_provider=james)
t

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:58290620765 -> alice:70287297970]
	-> (Wrapper)>[PointerTensor | me:37478403386 -> bob:16731539556]
	*crypto provider: james*

In [3]:
lr = torch.tensor([0.5]).fix_precision()

In [5]:
lr

(Wrapper)>FixedPrecisionTensor>tensor([500])

In [8]:
t.child.child.child, lr.child.child

([AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:58290620765 -> alice:70287297970]
 	-> (Wrapper)>[PointerTensor | me:37478403386 -> bob:16731539556]
 	*crypto provider: james*, tensor([500]))

In [10]:
t * lr

PureTorchTensorFoundError: 

In [9]:
t.child.child.child * lr.child.child

PureTorchTensorFoundError: 

In [ ]:
client_copy=[0]*2;
for epoch in range(1, 2):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        if batch_idx==0:
            bobmodel=model.copy().send(data.location) # <-- NEW: send the model to the rig
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = bobmodel(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step() 
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.data))

In [2]:
t = torch.tensor([1, 2, 3, 4]).send(bob)

In [4]:
t.shape == torch.Size([4])

torch.Size([4])

In [2]:
t = torch.tensor([1, 2, 3, 4])
x = t.share(bob, alice, crypto_provider=james)

In [3]:
bob._objects

{83918106758: tensor([1634305693511614491, 1798071659804866021, 4178895916032299915,
         1646802232034036872])}

In [4]:
alice._objects

{82335941561: tensor([-1634305693511614490, -1798071659804866019, -4178895916032299912,
         -1646802232034036868])}

In [5]:
ptr_to_sh = x.send(bob)
pointer = ptr_to_sh.remote_get()

70680666412


In [6]:
bob._objects

{83918106758: tensor([1634305693511614491, 1798071659804866021, 4178895916032299915,
         1646802232034036872]),
 70680666412: tensor([1, 2, 3, 4]),
 82335941561: tensor([-1634305693511614490, -1798071659804866019, -4178895916032299912,
         -1646802232034036868])}

In [7]:
alice._objects

{}

In [15]:
p = torch.tensor([1, 2, 3, 4]).send(bob)
p

(Wrapper)>[PointerTensor | me:56837727425 -> bob:79608462946]

In [17]:
p.copy().move(alice).get()

58170890915


tensor([1, 2, 3, 4])

In [2]:
t = torch.tensor([1, 2, 3, 4])
x = t.share(bob, alice, crypto_provider=james)

In [3]:
bob._objects

{14942537982: tensor([3097201650174364731, 3240755004145051928, 2983017260143293540,
         3483111410300018461])}

In [4]:
alice._objects

{99962979858: tensor([-3097201650174364730, -3240755004145051926, -2983017260143293537,
         -3483111410300018457])}

In [5]:
x2 = x.child.reconstruct()
print(x2)

91237032508
53842067586
[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:87001110324 -> bob:91237032508]
	-> (Wrapper)>[PointerTensor | me:53842067586 -> alice:53842067586]


In [6]:
bob._objects

{14942537982: tensor([3097201650174364731, 3240755004145051928, 2983017260143293540,
         3483111410300018461]),
 91237032508: tensor([1, 2, 3, 4]),
 99962979858: tensor([-3097201650174364730, -3240755004145051926, -2983017260143293537,
         -3483111410300018457])}

In [7]:
alice._objects

{53842067586: tensor([1, 2, 3, 4])}

In [ ]:
t = torch.tensor([1, 2, 3, 4])
x = t.share(bob, alice, crypto_provider=james)
print(x)
y = (x * x)

#z = y.get()

#assert (y == (t * t)).all() 72265325022

In [2]:
x = torch.tensor([1, 2, 3, 4, 5]).send(bob)
x.move(alice)

58202221436
48290417232


(Wrapper)>[PointerTensor | me:58202221436 -> alice:58202221436]

In [3]:
bob._objects

{}

In [4]:
alice._objects

{58202221436: tensor([1, 2, 3, 4, 5])}

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    @sy.method2plan
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)
    
    def your(self, x):
        return self.forward(self, x)

In [1]:
import torch

class MyModule(torch.jit.ScriptModule):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    @torch.jit.script_method
    def forward(self, input):
        if bool(input.sum() > 0):
            output = self.weight.mv(input)
        else:
            output = self.weight + input
        return output

my_script_module = MyModule(2, 3)

In [3]:
my_script_module.save("playground_model.pt")

In [8]:
my_script_module.graph

graph(%input : Tensor,
      %12 : Tensor):
  %2 : int = prim::Constant[value=1]()
  %3 : int = prim::Constant[value=0]()
  %4 : Tensor = aten::sum(%input)
  %5 : Tensor = aten::gt(%4, %3)
  %6 : bool = prim::Bool(%5)
  %output : Tensor = prim::If(%6)
    block0():
      %output.1 : Tensor = aten::mv(%12, %input)
      -> (%output.1)
    block1():
      %output.2 : Tensor = aten::add(%12, %input, %2)
      -> (%output.2)
  return (%output)

In [1]:
import torch
import torch.nn as nn
import syft as sy
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
import sys

In [2]:

a = torch.tensor([3, 2., 0], requires_grad=True)
b = torch.tensor([1., 2, 3], requires_grad=True)

a = a.fix_precision().share(alice, bob, local_autograd=True, crypto_provider=james)
b = b.fix_precision().share(alice, bob, local_autograd=True, crypto_provider=james)

c = a + b

#c.child.backward

send()
create prr True
send()
create prr True
send()
create prr True
send()
create prr True
AutogradTensor>[PointerTensor | me:69833126711 -> alice:83831799323]


AttributeError: can't set attribute

In [3]:
a

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>AutogradTensor>[PointerTensor | me:69741774145 -> alice:91163787230]
	-> (Wrapper)>AutogradTensor>[PointerTensor | me:58196706058 -> bob:9527471232]
	*crypto provider: james*

In [3]:
a = torch.tensor([3, 2., 0], requires_grad=True)
a = a.send(alice, local_autograd=True)

send()
create prr True


In [4]:
a

(Wrapper)>AutogradTensor>[PointerTensor | me:44497428780 -> alice:59716154869]

In [7]:
a.child.grad_fn #.grad_fn = 1

AttributeError: 'AutogradTensor' object has no attribute 'has_child'

In [4]:

a = torch.tensor([3, 2.0, 0], requires_grad=True)
b = torch.tensor([1, 2.0, 3], requires_grad=True)

a = a.send(alice, local_autograd=True)
b = b.send(alice, local_autograd=True)

a_torch = torch.tensor([3, 2.0, 0], requires_grad=True)
b_torch = torch.tensor([1, 2.0, 3], requires_grad=True)

c = a + b
c_torch = a_torch + b_torch

c.backward(torch.ones(c.shape).send(alice))
c_torch.backward(torch.ones(c_torch.shape))

assert (a.grad.get() == a_torch.grad).all()
assert (b.grad.get() == b_torch.grad).all()

In [2]:
workers = {'bob': bob, 'alice': alice}

In [3]:
# create tensor
x = torch.Tensor([1, 2])

# send tensor to bob and then pointer to alice
x_ptr = x.send(workers["bob"])
x_ptr_ptr = x_ptr.send(workers["alice"])

# ensure bob has tensor
assert x.id in workers["bob"]._objects

# delete pointer to pointer to tensor, which should automatically
# garbage collect the remote object on Bob's machine

print(sys.getrefcount(workers["bob"]._objects[x.id]))
del x_ptr_ptr

# ensure bob's object was garbage collected
assert x.id not in workers["bob"]._objects

2


In [2]:
# create tensor
x = torch.Tensor([1, 2])

# send tensor to bob and then pointer to alice
x_ptr = x.send(bob)

print('<<')
print(sys.getrefcount(bob._objects[x_ptr.id_at_location]))
print('>>')
x_ptr_ptr = x_ptr.send(alice)

# ensure bob has tensor
assert x.id in bob._objects

# delete pointer to pointer to tensor, which should automatically
# garbage collect the remote object on Bob's machine

print(sys.getrefcount(alice._objects[x_ptr_ptr.id_at_location]))
print(sys.getrefcount(bob._objects[x.id]))
del x_ptr_ptr

# ensure bob's object was garbage collected
assert x.id not in bob._objects

<<
2
>>
2
2
del
->done
ask DEL for (Wrapper)>[PointerTensor | alice:49902665511 -> bob:11907333849]
2
del
->done
ask DEL for tensor([1., 2.])
2


In [27]:
model = nn.Linear(2, 1)
model.weight = nn.Parameter(model.weight * 100)
model.weight

Parameter containing:
tensor([[11.2095, 31.0404]], requires_grad=True)

In [45]:
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
#model.bias = nn.Parameter(torch.tensor([[1.0]]))
print(model.weight)
model.fix_precision().share(bob, alice, crypto_provider=james)
model.weight.child.child.child.virtual_get()

Parameter containing:
tensor([[-1.,  2.]], requires_grad=True)


tensor([[0, 0]])

In [4]:
p = nn.Parameter(torch.tensor([[-1.0, 2]]))
o = p.sum()
o.backward()
p.grad -= p.grad
p

Parameter containing:
tensor([[0., 0.]], requires_grad=True)

In [3]:
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
#for p in model.parameters():
#    p.fix_precision_().share_(bob, alice, crypto_provider=james)
model.fix_precision().share(bob, alice, crypto_provider=james)

print(model.weight.child.child.child.virtual_get())

tensor([[-1000,  2000]])


In [34]:
p = nn.Parameter(torch.tensor([[-1.0, 2]]))
p.fix_precision_().share_(bob, alice, crypto_provider=james)
print(p.child.child.child.virtual_get())

tensor([[-1000,  2000]])


In [19]:
model.weight.child.child.child.virtual_get(), model.bias.child.child.child.virtual_get()

(tensor([[0, 0]]), tensor([0]))

In [2]:
t = torch.tensor([[1.0, 2]])
x = t.fix_prec().share(bob, alice, crypto_provider=james)
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([[-1.0]]))
model.fix_precision().share(bob, alice, crypto_provider=james)

y = model(x)

back = y.get().float_prec()
print(back)
assert back == torch.tensor([[2.0]])

HEY
MATMUL
tensor([[1000, 2000]])
tensor([[0, 0]])
tensor([[0]])
tensor([[0.]])


AssertionError: 

In [4]:
t = torch.tensor([[1.0, 2]])
x = t
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([[-1.0]]))

y = model(x)

back = y
print(back)

tensor([[2.]], grad_fn=<AddmmBackward>)


In [2]:

tensor = torch.tensor([1, 2, 3, 4.0])
ptr = tensor.send(bob)
r_ptr = torch.split(ptr, 2)
assert (r_ptr[0].get() == torch.tensor([1, 2.0])).all()

tensor = torch.tensor([1, 2, 3, 4.0])
ptr = tensor.send(bob)
max_value, argmax_idx = torch.max(ptr, 0)

resp (tensor([1., 2.]), tensor([3., 4.]))
rules (1, 1)
--
rules (1, 1)
resp torch.return_types.max(
values=tensor(4.),
indices=tensor(3))
rules 0
--
rules 0


TypeError: zip argument #2 must support iteration